### Battle simulation

In [21]:
# Model design
import agentpy as ap
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

# For dtypes
import numpy as np

In [3]:
class Soldier(ap.Agent):
    
    def setup(self):
        # team:
        # 'blue' -> 0
        # 'red' -> 1
        self.team = 'blue'
        # Initialize an attribute with a parameter
        self.health = 100
        self.damage = 12        
        
        # status:
        #   2 - alive and fighting (default)
        #   1 - surrendered and running away
        #   0 - dead
        self.status = 2
        
        
        # view range
        self.view_range = 2
        

        # self.endurance = 100
        # self.strength = 100
        # self.speed = 100
        # self.itd = 123
        # self.itp = 321
        
    
    def setup_position(self, space):
        self.pos = space.positions[self] 
        self.space = space
        self.neighbors = space.neighbors
        
        
    def attack(self, enemy):
        enemy.health -= self.damage
        # if soldier killed enemy
        if enemy.health <= 0:
            enemy.status = 0
    
    # TODO: fix
    def move(self, x, y):
        x_delta = random.randint(-1,1)
        y_delta = random.randint(-1,1)
        
        self.space.move_by(self, (x_delta, y_delta))
        nbs = self.neighbors(self, distance=self.view_range)
        return     
        
    def agent_method(self):
        # Define custom actions here
        pass

In [4]:
class BattleModel(ap.Model):
    
    def setup(self):
        # Create soldiers
        # count of soldiers in red team
        n_red_team = self.p['n_red_team']
        # count of soldiers in blue team
        n_blue_team = self.p['n_blue_team']
        # all soldiers
        n_soldiers = n_red_team + n_blue_team
        # create soldiers
        self.soldiers =  ap.AgentList(self, n_soldiers, Soldier)
        # soldier teams
        self.red_soldiers = []
        self.blue_soldiers = []
        
        # assign team for each soldier
        for i, soldier in enumerate(self.soldiers):
            # assign soldiers to red team
            if i < n_red_team:
                soldier.team = 'red'
                self.red_soldiers.append(soldier)
            # assign soldiers to blue team
            else:
                soldier.team = 'blue'
                self.blue_soldiers.append(soldier)
        
        # Create grid (battlefield)
        self.battlefield = ap.Grid(self, [self.p.size]*2, track_empty=True)      
        self.battlefield.add_agents(self.soldiers, random=True, empty=True)

        # Initiate a dynamic variable for all trees
        # status:
        #   2 - alive and fighting (default)
        #   1 - surrendered and running away
        #   0 - dead
        self.soldiers.status = 2 
        
        # Imform agents about their position
        for agent in self.soldiers:
            agent.setup_position(self.battlefield)
        
        # # Start a fire from the left side of the grid
        # unfortunate_trees = self.forest.agents[0:self.p.size, 0:2]
        # unfortunate_trees.condition = 1 
    
    def step(self):
        # Update posiotions of soliders
        
        
        # TODO: now whole red team has privilege to attack first
        # mixing the soldiers list before performing actions might help to level the playing field 

        # each red soldier attack
        for red_soldier in self.red_soldiers:
            # LOGGING
            # print("ajaj")
            # print(self.battlefield.neighbors(red_soldier, distance=1))
            # print(red_soldier)
            for neighbor in self.battlefield.neighbors(red_soldier, distance=1).to_list():
                if neighbor.team == 'blue':
                    # attack the first found neigbour from opposite team (break to not attack all neigbours but only the first one)
                    red_soldier.attack(neighbor)
                    break
            # TODO: soldier move after executing the attack
            # red_soldier.move()
            red_soldier.move(0, 0)
        
        # each blue soldier attack
        for blue_soldier in self.blue_soldiers:
            for neighbor in self.battlefield.neighbors(blue_soldier, distance=1).to_list():
                if neighbor.team == 'red':
                    # attack the first found neigbour from opposite team (break to not attack all neigbours but only the first one)
                    red_soldier.attack(neighbor)
                    break
            # TODO: soldier move after executing the attack
            blue_soldier.move(0, 0)
            
        

        # TODO: remove dead soldiers form self.soldiers and self.red/blue__team 
        # as they cannot interact with environment anymore
            
        # Stop simulation if there's only one team left
        if 0 in (len(self.red_soldiers), len(self.blue_soldiers)): 
            self.stop()
            
            
    def end(self):
        # Document a measure at the end of the simulation
        red_team_survivors = len(self.red_soldiers)
        blue_team_survivors = len(self.blue_soldier)
        self.report('Percentage of red team survivors', 
                    red_team_survivors / len(self.agents))
        self.report('Percentage of blue team survivors', 
                    blue_team_survivors / len(self.agents))
        self.report('Won team', 
                    'blue' if blue_team_survivors > 0 else 'red')

## Single-run animation

In [9]:
# Define parameters

parameters = {
    'n_red_team': 50, # number of team_1 soldiers
    'n_blue_team': 45, # number of team_2 soldiers
    'size': 30, 
    'steps': 12,
}

In [47]:
# Each cell of grid have status (alive, surrender, dead) and team (blue, red)
# To plot this, we need to convert this into tuple (status, team) and code each tuple as number
# But if-s are simpler
def create_cartesian_grid():
    # TODO: choose color for surrender units
    color_dict = {0:"#ffffff", 1:"#000000", 11:"#f0000", 12:"#ff0000", 21:"#0000ff", 22:"#ff0000"}
    
    status = model.battlefield.attr_grid('status')
    team = model.battlefield.attr_grid('team', [str])
    # team.shape == status.shape
    product = np.zeros(status.shape) # deafult: 0 (terrain)
    for x in range(product.shape[0]):
        for y in range(product.shape[1]):
            if status[x][y] == 0: # dead: 1
                product[x][y] = 1
            elif status[x][y] == 1 and team[x][y] == "blue": # blue and surrender: 11
                product[x][y] = 11
            elif status[x][y] == 1 and team[x][y] == "red": # red and surrender: 12
                product[x][y] = 12
            elif status[x][y] == 2 and team[x][y] == "blue": # blue and fighting: 21
                product[x][y] = 21 
            elif status[x][y] == 2 and team[x][y] == "red": # red and fighting: 22
                product[x][y] = 22
                
    return product, color_dict
    
    
# Create single-run animation withcustom colors
def animation_plot(model, ax):
    attr_grid, color_dict = create_cartesian_grid()
    #attr_grid = model.battlefield.attr_grid('status')
    #color_dict = {0:'#0048ff', 1:'#273746', 2:'#fc0303', None:'#423c3c'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Battle simulation\n"
                 # f"Time-step: {model.t}, Trees left: "
                 # f"{len(model.agents.select(model.agents.condition == 0))}"
                )

fig, ax = plt.subplots() 
model = BattleModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

## #TODO: Parameter sweep

In [7]:
# Prepare parameter sample
parameters = {
    'Tree density': ap.Range(0.2, 0.6), 
    'size': 100
}
sample = ap.Sample(parameters, n=30)

In [8]:
# Perform experiment
exp = ap.Experiment(ForestModel, sample, iterations=40)
results = exp.run()

NameError: name 'ForestModel' is not defined

In [ ]:
# Save and load data
results.save()
results = ap.DataDict.load('ForestModel')

In [ ]:
# Plot sensitivity
sns.set_theme()
sns.lineplot(
    data=results.arrange_reporters(), 
    x='Tree density', 
    y='Percentage of burned trees'
);